In [ ]:
%%capture
%pip install numpy==1.23.5
%pip install numba==0.57.1
%pip install matplotlib==3.6.3
%pip install shap==0.46.0
%pip install pandas==2.0.3
%pip install plotly==5.15.0
%pip install phik==0.12.3
%pip install scikit-learn==1.2.2
%pip install scipy==1.9.3
%pip install optuna==4.1.0
%pip install category-encoders==2.6.1
%pip install missingno==0.5.2
%pip install nbformat>=4.2.0
%pip install lightgbm==4.5.0
%pip install catboost==1.2.7
%pip install xgboost==2.1.4
%pip install Jinja2==3.1.2
%pip install statsmodels==0.14.4
%pip install requests==2.32.3

Active code page: 1251
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Active code page: 1251
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Active code page: 1251
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Active code page: 1251
  Using cached shap-0.46.0-cp311-cp311-win_amd64.whl.metadata (25 kB)
  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
Using cached shap-0.46.0-cp311-cp311-win_amd64.whl (456 kB)
Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl (11.6 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Active code page: 1251
  Using cached pandas-2.0.3-cp311-cp311-win_amd64.whl.metadata (18 kB)
Using cached pandas-2.0.3-cp311-cp311-win_amd64.whl (10.6 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.3
    Uninstalling pandas-2.2.3:
      Successfully uninstalled pandas-2.2.3
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import time
from copy import deepcopy
from itertools import cycle
from pprint import pprint
import re
from typing import Literal

import missingno as msno
import numpy as np
import optuna
import pandas as pd
import phik
import plotly
from category_encoders import (
    BinaryEncoder,
    CatBoostEncoder,
    LeaveOneOutEncoder,
    MEstimateEncoder,
    TargetEncoder
)
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from optuna.visualization import plot_param_importances
from plotly import express as px, graph_objects as go
from plotly.subplots import make_subplots
import requests
import shap
from sklearn.base import clone
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.feature_selection import (
    RFE,
    SelectKBest,
    mutual_info_regression
)
from sklearn.impute import IterativeImputer, KNNImputer, SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    MinMaxScaler,
    OneHotEncoder,
    OrdinalEncoder,
    RobustScaler,
    StandardScaler
)
from sklearn.tree import DecisionTreeRegressor
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from xgboost import XGBRegressor

ModuleNotFoundError: No module named 'missingno'

In [ ]:
pd.set_option('display.max_columns', None)
optuna.logging.set_verbosity(optuna.logging.WARNING)
plotly.offline.init_notebook_mode()

RANDOM_STATE = 42
optuna_sampler = optuna.samplers.TPESampler(seed=RANDOM_STATE)
state = np.random.RandomState(RANDOM_STATE)
cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
color_palette = cycle(px.colors.qualitative.Plotly)
optuna_storage = 'sqlite:///optuna.db'

_____
_____
## Загрузка данных, общая информация и первичная обработка

In [ ]:
train = pd.read_csv('train.csv')
train.head()